In [1]:
import os
import sys
sys.path.append("../..")
import phage_init

In [2]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x_data2_full.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x_data2_full.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y_data2_full.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y_data2_full.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean_data2_full.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std_data2_full.p"), "rb" ) )

In [3]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [4]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [5]:
mean_di     = mean_total[0:400]
mean_tri    = mean_total[400:8400]
mean_di_sc  = mean_total[8400:8449] 
mean_tri_sc = mean_total[8449:8792]
mean_tt     = mean_total[8792:]

In [6]:
std_di     = std_total[0:400]
std_tri    = std_total[400:8400]
std_di_sc  = std_total[8400:8449] 
std_tri_sc = std_total[8449:8792]
std_tt     = std_total[8792:]

In [7]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [8]:
#print(di_train.shape)
print(tt_train.shape)
#train_X =numpy.concatenate((tri_sc_train,tt_train),axis=1)
#test_X  =numpy.concatenate((tri_sc_test,tt_test),axis=1)
#train_X=di_train
#test_X=di_test
train_X=train_X_total
test_X=test_X_total
model_name='all_data2.h5'
print(train_X.shape)
print(test_X.shape)

(50000, 8)
(50000, 8800)
(7774, 8800)


In [9]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15805245970564989828
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16327172526759374775
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 42775304367825543
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8523306520487529328
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:3"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13090347602144100154
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:4"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7214734439596556010
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:5"
device_ty

In [10]:
print(train_X.shape[1])
f_num=train_X.shape[1]


8800


In [11]:
#model with 2 leyers of 100 LSTM neurons
model = Sequential()
opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8800)              77448800  
_________________________________________________________________
dropout_1 (Dropout)          (None, 8800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               1760200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)  

In [12]:
model.fit(train_X, train_Y, epochs=200,verbose=1, batch_size=2000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
50000/50000 [==============================] - 15s 309us/step - loss: 3.5554 - acc: 0.5880
Epoch 2/200
50000/50000 [==============================] - 12s 241us/step - loss: 0.5334 - acc: 0.8458
Epoch 3/200
50000/50000 [==============================] - 12s 242us/step - loss: 0.2504 - acc: 0.9269
Epoch 4/200
50000/50000 [==============================] - 12s 242us/step - loss: 0.1452 - acc: 0.9558
Epoch 5/200
50000/50000 [==============================] - 12s 247us/step - loss: 0.0992 - acc: 0.9701
Epoch 6/200
50000/50000 [==============================] - 12s 246us/step - loss: 0.0750 - acc: 0.9777
Epoch 7/200
50000/50000 [==============================] - 12s 247us/step - loss: 0.0619 - acc: 0.9817
Epoch 8/200
50000/50000 [==============================] - 12s 240us/step - loss: 0.0561 - acc: 0.9842
Epoch 9/200
50000/50000 [==============================] - 13s 252us/step - loss: 0.0480 - acc: 0.9861
Epoch 10/200
50000/50000 

50000/50000 [==============================] - 12s 243us/step - loss: 0.0224 - acc: 0.9946
Epoch 78/200
50000/50000 [==============================] - 12s 245us/step - loss: 0.0221 - acc: 0.9942
Epoch 79/200
50000/50000 [==============================] - 12s 246us/step - loss: 0.0216 - acc: 0.9948
Epoch 80/200
50000/50000 [==============================] - 12s 248us/step - loss: 0.0189 - acc: 0.9951
Epoch 81/200
50000/50000 [==============================] - 12s 240us/step - loss: 0.0200 - acc: 0.9947
Epoch 82/200
50000/50000 [==============================] - 12s 248us/step - loss: 0.0195 - acc: 0.9950
Epoch 83/200
50000/50000 [==============================] - 12s 243us/step - loss: 0.0186 - acc: 0.9952
Epoch 84/200
50000/50000 [==============================] - 12s 248us/step - loss: 0.0159 - acc: 0.9954
Epoch 85/200
50000/50000 [==============================] - 12s 245us/step - loss: 0.0174 - acc: 0.9953
Epoch 86/200
50000/50000 [==============================] - 12s 249us/step - 

50000/50000 [==============================] - 12s 244us/step - loss: 0.0162 - acc: 0.9963
Epoch 156/200
50000/50000 [==============================] - 12s 242us/step - loss: 0.0162 - acc: 0.9959
Epoch 157/200
50000/50000 [==============================] - 12s 238us/step - loss: 0.0162 - acc: 0.9960
Epoch 158/200
50000/50000 [==============================] - 12s 248us/step - loss: 0.0169 - acc: 0.9960
Epoch 159/200
50000/50000 [==============================] - 13s 250us/step - loss: 0.0177 - acc: 0.9962
Epoch 160/200
50000/50000 [==============================] - 12s 242us/step - loss: 0.0186 - acc: 0.9961
Epoch 161/200
50000/50000 [==============================] - 12s 244us/step - loss: 0.0195 - acc: 0.9955
Epoch 162/200
50000/50000 [==============================] - 12s 242us/step - loss: 0.0190 - acc: 0.9958
Epoch 163/200
50000/50000 [==============================] - 12s 246us/step - loss: 0.0177 - acc: 0.9953
Epoch 164/200
50000/50000 [==============================] - 12s 236u

In [13]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

7774/7774 [==============================] - 2s 276us/step
Accuracy: 91.36%


In [14]:
#you can save the as a file so you dont have to train it every time
model.save( os.path.join(phage_init.model_dir,model_name))

In [15]:
from sklearn.metrics import classification_report


test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
test_Y_predicted = model.predict_classes(test_X)
labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
#print (test_Y_index)

                   precision    recall  f1-score   support

     Major capsid       0.96      0.95      0.95      1836
     Minor capsid       0.71      0.53      0.61       105
        Baseplate       0.90      0.85      0.87       308
       Major tail       0.86      0.91      0.88       423
       Minor tail       0.91      0.80      0.85       499
           Portal       0.93      0.96      0.95      2473
       Tail fiber       0.75      0.75      0.75       412
       Tail shaft       0.94      0.94      0.94       677
           Collar       0.74      0.80      0.77       149
Head-Tail joining       0.90      0.92      0.91       892

      avg / total       0.91      0.91      0.91      7774



In [16]:
from collections import Counter
zz=Counter(test_Y_index)
sample_w=[zz[i] for i in range(0,10,1)]
print(zz)
print(sample_w)
print()

Counter({5: 2473, 0: 1836, 9: 892, 7: 677, 4: 499, 3: 423, 6: 412, 2: 308, 8: 149, 1: 105})
[1836, 105, 308, 423, 499, 2473, 412, 677, 149, 892]



In [17]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

CM=confusion_matrix(test_Y_index, test_Y_predicted)
CM_n=CM/numpy.array(sample_w)[:,None]
scale_up=1.3
plt.figure(figsize=[6.4*scale_up, 4.8*scale_up])
plt.imshow(CM_n, interpolation='nearest')
plt.title('CM')
plt.colorbar()
tick_marks = numpy.arange(len(labels_names))
plt.xticks(tick_marks, labels_names, rotation=90)
plt.yticks(tick_marks, labels_names)
fmt = '.2f'
for i, j in itertools.product(range(CM_n.shape[0]), range(CM_n.shape[1])):
        plt.text(j, i, format(CM_n[i, j], fmt),horizontalalignment="center",verticalalignment='center',
                color="white" if CM_n[i, j] < 0.25 else "black")
plt.show()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


<Figure size 832x624 with 2 Axes>